## Importations des tables orders et users

In [8]:
import pandas as pd
import duckdb as db

# Création de la table users
data_users = {
    'user_id': [1, 2, 3, 4],
    'user_name': ['Alice', 'Bob', 'Charlie', 'Diana']
}

users = pd.DataFrame(data_users)

# Création de la table orders
data_orders = {
    'order_id': [101, 102, 103, 104, 105],
    'user_id': [1, 2, 1, 3, 5],  # Notez que user_id 5 n'existe pas dans df_users
    'product': ['Shirt', 'Shoes', 'Hat', 'Shirt', 'Bag'],
    'amount': [20, 50, 15, 20, 25]
}

orders = pd.DataFrame(data_orders)



## Les jointures

Les jointures en SQL permettent de joindre, de fusionner deux ou plusieurs tables entre elles; des tables qui sont liés entre elles par des clés soit primaire ou étrangère

Une table contient une clé primaire qui permet d’identifier de manière unique chaque enregistrement ou ligne de la table;

Et une clé étrangère qui est la clé primaire d’une autre table.

Je prends par exemple deux tables quelconques, la table Commande et la table client.
La table client contient les colonnes telles que le nom, prénom, du client ainsi que la clé primaire qui permet d’identifier de manière unique chaque client.
Et la table commande contient les informations sur chaque commande tel que le nom du produit, le prix de la commande, mais aussi l'identifiant du client qui a passé la commande et cet identifiant est la clé étrangère, car fait référence à la clé primaire de la table client
vous savez pourquoi c’est la table commande qui porte la clé étrangère. 

Les types de jointures

- inner join
- Left join
- full join
- cross join
- union join
- self  join

In [6]:
users.head(10)

,user_id,user_name
0,1,Alice
1,2,Bob
2,3,Charlie
3,4,Diana


In [5]:
orders.head(10)

,order_id,user_id,product,amount
0,101,1,Shirt,20
1,102,2,Shoes,50
2,103,1,Hat,15
3,104,3,Shirt,20
4,105,5,Bag,25


### inner join 

L'Inner Join combine les lignes des deux tables où il y a une correspondance sur la clé de jointure.

In [10]:
query = """
SELECT users.user_id, users.user_name, orders.order_id, orders.product, orders.amount
FROM users
INNER JOIN orders ON users.user_id = orders.user_id;
"""

db.sql(query)

┌─────────┬───────────┬──────────┬─────────┬────────┐
│ user_id │ user_name │ order_id │ product │ amount │
│  int64  │  varchar  │  int64   │ varchar │ int64  │
├─────────┼───────────┼──────────┼─────────┼────────┤
│       1 │ Alice     │      101 │ Shirt   │     20 │
│       2 │ Bob       │      102 │ Shoes   │     50 │
│       1 │ Alice     │      103 │ Hat     │     15 │
│       3 │ Charlie   │      104 │ Shirt   │     20 │
└─────────┴───────────┴──────────┴─────────┴────────┘

### left join

Le Left Join retourne toutes les lignes de la table de gauche (users), ainsi que les lignes correspondantes de la table de droite (orders). Si aucune correspondance n'est trouvée, les résultats de la table de droite contiendront des NULL.

In [11]:
query = """
SELECT users.user_id, users.user_name, orders.order_id, orders.product, orders.amount
FROM users
LEFT JOIN orders ON users.user_id = orders.user_id;

"""

db.sql(query)

┌─────────┬───────────┬──────────┬─────────┬────────┐
│ user_id │ user_name │ order_id │ product │ amount │
│  int64  │  varchar  │  int64   │ varchar │ int64  │
├─────────┼───────────┼──────────┼─────────┼────────┤
│       1 │ Alice     │      103 │ Hat     │     15 │
│       2 │ Bob       │      102 │ Shoes   │     50 │
│       3 │ Charlie   │      104 │ Shirt   │     20 │
│       1 │ Alice     │      101 │ Shirt   │     20 │
│       4 │ Diana     │     NULL │ NULL    │   NULL │
└─────────┴───────────┴──────────┴─────────┴────────┘

### full Join

Le Full Join (ou Full Outer Join) retourne toutes les lignes lorsque les conditions sont remplies dans l'une ou l'autre des tables. Les lignes non appariées dans l'une ou l'autre table apparaîtront avec des NULL pour les colonnes de la table qui ne correspond pas.


In [17]:
query = """
SELECT users.user_id, users.user_name, orders.order_id, orders.product, orders.amount
FROM users
FULL JOIN orders ON users.user_id = orders.user_id;


"""

db.sql(query)

┌─────────┬───────────┬──────────┬─────────┬────────┐
│ user_id │ user_name │ order_id │ product │ amount │
│  int64  │  varchar  │  int64   │ varchar │ int64  │
├─────────┼───────────┼──────────┼─────────┼────────┤
│       1 │ Alice     │      103 │ Hat     │     15 │
│       2 │ Bob       │      102 │ Shoes   │     50 │
│       3 │ Charlie   │      104 │ Shirt   │     20 │
│       1 │ Alice     │      101 │ Shirt   │     20 │
│       4 │ Diana     │     NULL │ NULL    │   NULL │
│    NULL │ NULL      │      105 │ Bag     │     25 │
└─────────┴───────────┴──────────┴─────────┴────────┘

### cross join


In [19]:
query = """
SELECT *
FROM users
CROSS JOIN orders;

"""

db.sql(query)

┌─────────┬───────────┬──────────┬─────────┬─────────┬────────┐
│ user_id │ user_name │ order_id │ user_id │ product │ amount │
│  int64  │  varchar  │  int64   │  int64  │ varchar │ int64  │
├─────────┼───────────┼──────────┼─────────┼─────────┼────────┤
│       1 │ Alice     │      101 │       1 │ Shirt   │     20 │
│       1 │ Alice     │      102 │       2 │ Shoes   │     50 │
│       1 │ Alice     │      103 │       1 │ Hat     │     15 │
│       1 │ Alice     │      104 │       3 │ Shirt   │     20 │
│       1 │ Alice     │      105 │       5 │ Bag     │     25 │
│       2 │ Bob       │      101 │       1 │ Shirt   │     20 │
│       2 │ Bob       │      102 │       2 │ Shoes   │     50 │
│       2 │ Bob       │      103 │       1 │ Hat     │     15 │
│       2 │ Bob       │      104 │       3 │ Shirt   │     20 │
│       2 │ Bob       │      105 │       5 │ Bag     │     25 │
│       3 │ Charlie   │      101 │       1 │ Shirt   │     20 │
│       3 │ Charlie   │      102 │      

### union

L'Union combine les résultats de deux requêtes SQL. Chaque requête doit retourner le même nombre de colonnes, et les types de données correspondants doivent être compatibles.
Les doublons sont supprimés

In [22]:
query = """
SELECT user_id, user_name FROM users
UNION
SELECT user_id, user_name FROM users;

"""

db.sql(query)

┌─────────┬───────────┐
│ user_id │ user_name │
│  int64  │  varchar  │
├─────────┼───────────┤
│       2 │ Bob       │
│       1 │ Alice     │
│       3 │ Charlie   │
│       4 │ Diana     │
└─────────┴───────────┘

### self join

Un Self Join est une jointure d'une table avec elle-même. Cela peut être utile pour comparer les lignes au sein de la même table.
Ces jointures sont utilisées lorsqu’une table possède une clé primaire et une clé étrangère à la fois.

Exemple : Hiérarchie des employés

#### table employees

In [25]:
# Création de la table employees
data_employees = {
    'emp_id': [1, 2, 3, 4, 5],
    'emp_name': ['John', 'Jane', 'Joe', 'Anna', 'Mike'],
    'manager_id': [None, 1, 1, 2, 2]
}

employees = pd.DataFrame(data_employees)

#### requete self join

In [26]:
employees.head()

,emp_id,emp_name,manager_id
0,1,John,NaN
1,2,Jane,1.0
2,3,Joe,1.0
3,4,Anna,2.0
4,5,Mike,2.0


In [24]:
query =""" 
SELECT e1.emp_name AS employee, e2.emp_name AS manager
FROM employees e1
LEFT JOIN employees e2 ON e1.manager_id = e2.emp_id;
"""
db.sql(query)

┌──────────┬─────────┐
│ employee │ manager │
│ varchar  │ varchar │
├──────────┼─────────┤
│ Jane     │ John    │
│ Joe      │ John    │
│ Anna     │ Jane    │
│ Mike     │ Jane    │
│ John     │ NULL    │
└──────────┴─────────┘